In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm

# 
import numpy as np
import os
from tqdm import trange
import parmap
import glob
from sklearn.decomposition import PCA
#import umap

from tqdm import tqdm
import sleap

import sklearn.experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 
from scipy.io import loadmat
import scipy
import scipy.ndimage
# 

#import numba
#from numba import jit
import parmap

# 
from track import track
from convert import convert
from ethogram import ethogram

Autosaving every 180 seconds


In [ ]:
def score_id(labels_gt, labels_pr):
    """Compute ID metrics for two labels."""
    n_gt, n_pr = 0, 0
    tp, fp, fn  = 0, 0, 0
    gt_without_id = 0
    for lf_gt, lf_pr in zip(labels_gt, labels_pr):
        assert lf_gt.frame_idx == lf_pr.frame_idx  # weak check but it's what we have until we fix the video metadata serialization

        n_gt += len(lf_gt)
        n_pr += len(lf_pr)

        positive_pairs, false_negatives = sleap.nn.evals.match_instances(lf_gt, lf_pr)
        fn += len(false_negatives)  # gt instances that were not found
        fp += len(lf_pr) - len(positive_pairs)  # extra pr instances
        
        for inst_gt, inst_pr, _ in positive_pairs:
            if inst_gt.track is None:  # gt not id labeled
                gt_without_id += 1
                continue
            if inst_pr.track is None:  # pr not id labeled
                fn += 1
                continue
            if inst_pr.track.name == inst_gt.track.name:
                tp += 1
            else:
                fp += 1
    # 
    return {"n_gt": n_gt,
            "n_pr": n_pr,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "gt_without_id": gt_without_id,
            "acc": tp / n_gt}


In [ ]:
################################################################
############# MAKE TRAINING PKGs ON WORKSTATION #2 #############
################################################################
import sleap
import numpy as np

sleap.disable_preallocation()
sleap.versions()

# 
cohort = 'cohort1'
time_of_day = 'day'
root_dir = os.path.join('/home/cat/code/sleap/',cohort,time_of_day,
                        'cross_validation/')

# Load sleap pkg file containing labeled data and images
labels = sleap.load_file(root_dir+'/'+cohort+"_"+time_of_day+'.pkg.slp')

# Extract just user labeled frames
user_labels = labels.extract(labels.user_labeled_frame_inds)

# Subsample labels
n_samples = [100,200,300,400,450]

# 
for n_sample in n_samples:
    idx = np.random.choice(np.arange(n_sample), n_sample, replace=False)  # This extracts 100 unique random ids from 500 total
    train_user_labels = user_labels.extract(idx)

    # Save index
    np.save(root_dir + '/idx_'+cohort+"_"+time_of_day+'_subsample_'+str(n_sample)+'.npy', idx)

    # Save new .pkg file
    train_user_labels.save(root_dir + "/"+cohort+"_"+time_of_day+"_"+"_subsample_"+str(n_sample)+".pkg.slp", 
                           with_images = True)


In [ ]:
############################################################
############# MAKE TEST PKGs ON WORKSTATION #1 #############
############################################################
import sleap
import numpy as np

sleap.disable_preallocation()
sleap.versions()

# 
cohort = 'cohort1'
time_of_day = 'day'
root_dir = os.path.join('/home/cat/data/',cohort,time_of_day)


# Load sleap pkg file containing labeled data and images
labels = sleap.load_file(root_dir+'/'+cohort+"_"+time_of_day+'.pkg.slp')

# Extract just user labeled frames
user_labels = labels.extract(labels.user_labeled_frame_inds)

# Subsample labels
n_samples = [100,200,300,400,450]

# 
for n_sample in n_samples:
    idx_train = root_dir + '/idx_'+cohort+'_'+time_of_day+'_subsample_'+str(n_sample)+'.npy' # load saved training set indexes
    
    # delete the training indexes from the total # of frames (500) to get all potential training indexes
    idx_test = np.delete(np.arange(500), idx_train)
    
    # subselect only 50 training
    idx_test = np.random.choice(idx_test, 50, replace=False)
        
    test_user_labels = user_labels.extract(idx)

    # Save test .pkg file
    test_user_labels.save(root_dir + cohort+"_"+time_of_day+"_subsample_"+str(n_sample)+"_test.pkg.slp", 
                          with_images = True)

In [139]:

####################################################
################# EVALUTE ERRORS ###################
####################################################

def get_matrices(cohort, labels_gt, labels_pr):

#     if cohort == 'cohort1':
#         tp = {'male':0, 
#               'female':0,
#               'pup shaved':0,
#               'pup unshaved':0
#               }

#         fp = {'male':0, 
#                'female':0,
#                'pup shaved':0,
#                'pup unshaved':0
#               }

#         fn = {'male':0, 
#                'female':0,
#                'pup shaved':0,
#                'pup unshaved':0
#               }

#         n_labeled = {'male':0, 
#                        'female':0,
#                        'pup shaved':0,
#                        'pup unshaved':0
#                       }
    else:
        tp = {'male':0, 
          'female':0,
          'pup1':0,
          'pup2':0,
          'pup3':0,
          'pup4':0
          }

        fp = {'male':0, 
          'female':0,
          'pup1':0,
          'pup2':0,
          'pup3':0,
          'pup4':0
          }
        
        fn = {'male':0, 
          'female':0,
          'pup1':0,
          'pup2':0,
          'pup3':0,
          'pup4':0
          }
        
        n_labeled = {'male':0, 
          'female':0,
          'pup1':0,
          'pup2':0,
          'pup3':0,
          'pup4':0
          }
    
    
    # loop over all labeled frames
    for lf_gt, lf_pr in zip(labels_gt, labels_pr):
        assert lf_gt.frame_idx == lf_pr.frame_idx  # weak check but it's what we have until we fix the video metadata serialization
        # n_gt += len(lf_gt)
        # n_pr += len(lf_pr)

        positive_pairs, false_negatives = sleap.nn.evals.match_instances(lf_gt, lf_pr)
        # FN = len(false_negatives)  # gt instances that were not found
        # FP = len(lf_pr) - len(positive_pairs)  # extra pr instances

                
        # 
        for inst_gt, inst_pr, _ in positive_pairs:
           
            # 
            if inst_gt.track is None:  # gt not id labeled
                #gt_without_id += 1
                continue
                
            # 
            if inst_pr.track is None:  # pr not id labeled
                fn[inst_pr.track.name]+=1
                continue

            # 
            if inst_pr.track.name == inst_gt.track.name:
                tp[inst_pr.track.name]+=1

            else:
                fp[inst_pr.track.name]+=1
                pass 
        
        # add animal specific 
        for inst_gt in false_negatives:
            if inst_gt.track is not None:
                fn[inst_gt.track.name]+=1
           
    return tp, fp, fn


########################################################################

import sleap
import os
import pickle
from tqdm import trange

sleap.disable_preallocation()
sleap.versions()

# 
cohort = 'cohort1'
time_of_day = 'night'
root_dir = os.path.join('/media/cat/256GB/dan/',cohort,time_of_day)

#
if cohort=='cohort1':
    tracks = ['male', 'female', 'pup shaved', 'pup unshaved']
    n_animals=4
else:
    tracks = ['male', 'female', 'pup1', 'pup2', 'pup3','pup4']
    n_animals=6

    
# load gt labels
n_samples = [100, 200, 300, 400, 450]

#
tp_array = np.zeros((5,10,n_animals),'int32')
fp_array = np.zeros((5,10,n_animals),'int32')
fn_array = np.zeros((5,10,n_animals),'int32')

#
for n,n_sample in enumerate(n_samples):
    
    # 
    for k in trange(10):
        
        #
        labels_gt = sleap.load_file(root_dir+"/test_"+str(k)+"_subsample_"+str(n_sample)+".pkg.slp")
        labels_pr = sleap.load_file(root_dir+"/test_"+str(k)+"_subsample_"+str(n_sample)+".pkg.slp.predictions.slp")

        # 
        tp, fp, fn = get_matrices(cohort, labels_gt, labels_pr)

        # 
        for a in range(n_animals):
            tp_array[n,k,a] = tp[tracks[a]]
            fp_array[n,k,a] = fp[tracks[a]]
            fn_array[n,k,a] = fn[tracks[a]]

        if n_sample==450:
            break
            
print (" DONE ")

  0%|          | 0/10 [00:00<?, ?it/s]

SLEAP: 1.1.1
TensorFlow: 2.3.1
Numpy: 1.18.5
Python: 3.7.0
OS: Linux-5.4.0-84-generic-x86_64-with-debian-buster-sid


  0%|          | 0/10 [00:00<?, ?it/s]

 DONE 


In [141]:
#####################################
#####################################
#####################################

def plot_scatter(temp, clr, label):
    mean = np.mean(temp)/50.
    std = np.std(temp)/50.

    plt.scatter(n_samples[n], mean, label=label, c=clr)
    plt.errorbar(n_samples[n], mean, yerr=std,
                c=clr)#, xlolims=True, label='xlolims=True')
    return mean
        
        
n_s = np.arange(5)

if cohort=='cohort1':
    n_animals=4
else:
    n_animals=6

labels = ['true pos','false pos','false neg']
for n in n_s:

    #
    for a in range(n_animals):
        ax=plt.subplot(2,n_animals//2,a+1)

        temp = tp_array[n,:,a]
        if n==4:
            temp = temp[0]

        clr='black'
        mean = plot_scatter(temp,clr, labels[0])
        
        # false positive
        temp2 = fp_array[n,:,a]
        if n==4:
            temp2 = temp2[0]
        clr='red'
        mean = plot_scatter(temp2,clr, labels[1])

        
        # false negative
        temp2 = fn_array[n,:,a]
        if n==4:
            temp2 = temp2[0]
        clr='blue'
        mean = plot_scatter(temp2,clr, labels[2])
        
        # 
        plt.ylim(0,1.0)
        plt.xlim(0,1000)
        plt.title(tracks[a])
    # #
        if n==0 and a==0:
            plt.legend()
plt.suptitle(cohort+ " "+time_of_day)
plt.show()
        




In [73]:
#################################################
#################################################
#################################################
cohort = 'cohort1'
time_of_day = 'day'
root_dir = os.path.join('/media/cat/256GB/dan/',cohort,time_of_day)

n_samples = [100, 200,300,400,450]
for n_sample in n_samples:

    for k in range(10):

        labels_gt = sleap.load_file(root_dir+"/test_"+str(k)+"_subsample_"+str(n_sample)+".pkg.slp")
        labels_pr = sleap.load_file(root_dir+"/test_"+str(k)+"_subsample_"+str(n_sample)+".pkg.slp.predictions.slp")

        
        
        

In [74]:
#################################################
#################################################
#################################################


len labels:  Labels(labeled_frames=50, videos=11, skeletons=1, tracks=4) 50


In [20]:
#######################################################
############# PLOT EVALUATION RESULTS #################
#######################################################
import pickle
def load_obj(name ):
    with open(name, 'rb') as f:
        return pickle.load(f)

n_samples = [100,200,300,400,450]
fig=plt.figure()

cohorts = ['cohort1','cohort2']
times = ['day','night']

cohorts = ['cohort1']
times = ['night']

ctr=0
for cohort in cohorts:
    for time_of_day in times:
        ax=plt.subplot(2,2,ctr+1)
        fits = []

        for n_sample in n_samples:

            acc = []
            for k in range(10):

                res = load_obj('/media/cat/256GB/dan/'+cohort+'/'+time_of_day+'/res_'+str(k)+'_subsample_'+str(n_sample)+'.pkl')

                temp = res['acc']
                print (k, temp)
                acc.append(temp)
                
                if n_sample==450:
                    for p in range(9):
                        acc.append(temp)
                    break
            # 
            mean = np.mean(acc)
            std = np.std(acc)
            plt.scatter(n_sample, mean, label=str(n_sample))

            plt.errorbar(n_sample, mean, yerr=std )#, xlolims=True, label='xlolims=True')

            fits.append(mean)
            


        #
        print (n_samples)
        print (fits)
        p = np.poly1d(np.polyfit(n_samples, fits, 1))

        x = np.arange(3000)

        px = p(x)
        plt.plot(x,px,label='linear fit')

        #
        diffs = px-0.8
        idx = np.argmin(np.abs(diffs))
        plt.plot([0,5000],[0.8,0.8],'--',c='grey',label='80% acc')
        plt.plot([idx,idx],[0,1],'--',c='grey')


        diffs = px-0.9
        idx = np.argmin(np.abs(diffs))
        plt.plot([0,5000],[0.9,0.9],'--',c='lightblue',label='90% acc')
        plt.plot([idx,idx],[0,1],'--',c='lightblue')



        plt.legend()

        # 
        plt.ylim(0,1)
        plt.xlim(0,1000)
        plt.ylabel("Accuracy \n(50 validation frames; std shown)")
        plt.xlabel("# of training frames")
        plt.title("Cohort 1 (4 gerbils) - daytime videos",fontsize=16)

    #
        plt.title(cohort+ " "+time_of_day)
        ctr+=1
plt.show()

0 0.5329341317365269
1 0.5680473372781065
2 0.5886075949367089
3 0.5963855421686747
4 0.5802469135802469
5 0.5301204819277109
6 0.5031055900621118
7 0.5534591194968553
8 0.5060240963855421
9 0.6144578313253012
0 0.5859872611464968
1 0.6012658227848101
2 0.5414012738853503
3 0.5636363636363636
4 0.525974025974026
5 0.524390243902439
6 0.515527950310559
7 0.5032679738562091
8 0.6078431372549019
9 0.5652173913043478
0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
0 0.7161290322580646
1 0.6776315789473685
2 0.7092198581560284
3 0.6797385620915033
4 0.7077922077922078
5 0.7328767123287672
6 0.7074829931972789
7 0.6912751677852349
8 0.6870748299319728
9 0.68
0 0.8823529411764706
[100, 200, 300, 400, 450]
[0.5573388638897785, 0.5534511444055503, 0.0, 0.6989220942488427, 0.8823529411764707]


In [14]:
print (res)

{'n_gt': 239, 'n_pr': 178, 'tp': 153, 'fp': 20, 'fn': 10, 'gt_without_id': 5, 'acc': 0.6401673640167364}


In [17]:
############################################
########### LOAD AND SPLIT DATA ############
############################################

# Load sleap pkg file containing labeled data and images
labels = sleap.load_file('/media/cat/256GB/dan/cohort1/cohort1_day_only_labeled_frames.pkg.slp')

print ("labels.count: ", labels.count)

for k in range(len(labels.videos)):
    #print (k, labels.labels[k])
    print (k, labels.videos[k])
 
print (labels.videos[1].shape)

print ('')

idx_train = np.arange(450)
labels_train = labels.extract(idx_train) 
print (labels_train)

labels.count:  <bound method Sequence.count of Labels(labeled_frames=2397, videos=12, skeletons=1, tracks=4)>
0 Video(filename=/media/cat/256GB/dan/cohort1/cohort1_day_only_labeled_frames.pkg.slp, shape=(0, 1024, 1280, 3), backend=HDF5Video)
1 Video(filename=/media/cat/256GB/dan/cohort1/cohort1_day_only_labeled_frames.pkg.slp, shape=(20, 1024, 1280, 3), backend=HDF5Video)
2 Video(filename=/media/cat/256GB/dan/cohort1/cohort1_day_only_labeled_frames.pkg.slp, shape=(50, 1024, 1280, 3), backend=HDF5Video)
3 Video(filename=/media/cat/256GB/dan/cohort1/cohort1_day_only_labeled_frames.pkg.slp, shape=(90, 1024, 1280, 3), backend=HDF5Video)
4 Video(filename=/media/cat/256GB/dan/cohort1/cohort1_day_only_labeled_frames.pkg.slp, shape=(115, 1024, 1280, 3), backend=HDF5Video)
5 Video(filename=/media/cat/256GB/dan/cohort1/cohort1_day_only_labeled_frames.pkg.slp, shape=(50, 1024, 1280, 3), backend=HDF5Video)
6 Video(filename=/media/cat/256GB/dan/cohort1/cohort1_day_only_labeled_frames.pkg.slp, shape

In [ ]:

#        #
# labels_train = labels.extract(idx_train, copy=True)
# labels_val = labels.extract(idx_val, copy=True)
# labels_test = labels.extract(idx_test, copy=True)


# # Generate a 0.8/0.1/0.1 split
# labels_train, labels_test = labels.split(n=0.9)
# # labels_val, labels_test = labels_val_test.split(n=0.5)


# # # Save with images
# # labels_train.save("train.pkg.slp", with_images=True)
# # #labels_val.save("val.pkg.slp", with_images=True)
# # labels_test.save("test.pkg.slp", with_images=True)


# # # Reload and check the grond truh labels in each of these packages:
# # labels = sleap.load_file("train.pkg.slp")
# labels....

In [6]:
import numpy as np
from sklearn.model_selection import KFold
X = np.arange(100)
y = np.zeros(100)
kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(X)

print(kf)

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    

KFold(n_splits=10, random_state=None, shuffle=True)
TRAIN: [ 0  2  3  4  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 24 25 26 27
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 44 45 46 47 48 50 51 52 53 54
 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 74 75 76 77 78 80
 81 82 83 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99] TEST: [ 1  5 17 23 28 43 49 73 79 84]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24
 25 27 28 29 30 31 32 33 34 36 37 38 39 40 42 43 44 45 46 47 48 49 50 51
 52 54 55 56 57 58 59 60 61 62 63 64 65 68 69 70 71 72 73 74 75 76 77 78
 79 80 81 82 83 84 85 86 87 88 89 91 92 93 94 95 97 98] TEST: [13 26 35 41 53 66 67 90 96 99]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 16 17 19 20 21 22 23 24 25 26
 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
 51 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 69 70 71 72 73 75 76 77
 78 79 80 81 82 84 85 86 87 88 89 90 91 93 94 96 98 99] TEST: [11 15 18 52 68 74 8

In [11]:
#############################
### CUSTOM SPLIT FUNCTION ###
#############################

# load sleap label file
# labels = sleap.load_file(...)

#
n_tot_labels = 500      # total # of human labels

n_labels = 100          # from 100 to 500 labels 

n_validation = 10

val_ratio = 0.9

n_train = int(n_labels*val_ratio)
n_test = int(n_labels*(1-val_ratio))
    
# shuffled res
idx = np.random.choice(np.arange(n_tot_labels),n_tot_labels, replace=False)    

# 
for k in range(n_validation):
    idx_train = idx[n_test*k:n_test*k+n_train]
    idx_test = idx[n_test*k+n_train:n_test*(k+1)+n_train]
    print ("idx_train:", idx_train)
    print ("idx_test: ", idx_test)
    
#     # 
#     labels_train = labels.extract(idx_train, copy=True)
#     labels_test = labels.extract(idx_test, copy=True)

#     # Save with images
#     labels_train.save("train_"+str(ctr)+".pkg.slp", with_images=True)
#     labels_test.save("test_"+str(ctr)+".pkg.slp", with_images=True)

    
    

idx_train: [411 271 127 200   2 422 360 106 368 392 464 400 341  12 386 166  49 245
  88 218 275 111  78  77  95 205 138 348  74  57 153 165 281 171 371   5
  45  43   0   7  65  92 300 316 402 337 370 135 454 363 235 163  22 263
  64 314 190  70 322 162 172  97  68   3 262 351 320 404 486 283 497 425
 108 330 434 141 164 357 498 143 145 173 239 419   1 352 225  20 113 313]
idx_test:  [ 25 397 250 191 215  83  18 212 276]
idx_train: [392 464 400 341  12 386 166  49 245  88 218 275 111  78  77  95 205 138
 348  74  57 153 165 281 171 371   5  45  43   0   7  65  92 300 316 402
 337 370 135 454 363 235 163  22 263  64 314 190  70 322 162 172  97  68
   3 262 351 320 404 486 283 497 425 108 330 434 141 164 357 498 143 145
 173 239 419   1 352 225  20 113 313  25 397 250 191 215  83  18 212 276]
idx_test:  [267 181  90  37 347   4 439 323 130]
idx_train: [ 88 218 275 111  78  77  95 205 138 348  74  57 153 165 281 171 371   5
  45  43   0   7  65  92 300 316 402 337 370 135 454 363 235 163

In [ ]:
#########################################################
######### TRAIN SLEAP ON A TRAINING PKG DATASET #########
#########################################################

# comamnd line run:
command = 'sleap-train '+'train.pkg.slp'

os.command(command)

In [ ]:
###########################################
######### PREDICT ON THE TEST SET #########
###########################################

# comamnd line run:
command = 'sleap-train '+'test.pkg.slp'

os.command(command)



In [ ]:
###########################################
######### PREDICT ON THE TEST SET #########
###########################################

# Reload and check the grond truh labels in each of these packages:
labels = sleap.load_file("train.pkg.slp")
labels....




In [ ]:
###############################################################
### REDEFINE SPLIT FUNCTION TO ALLOW FOR PROPER XVALIDATION ###
###############################################################

def split( 
         self, n: Union[float, int], copy: bool = True 
     ) -> Tuple["Labels", "Labels"]: 
         """Split labels randomly. 
  
         Args: 
             n: Number or fraction of elements in the first split. 
             copy: If `True` (the default), return copies of the labels. 
  
         Returns: 
             A tuple of `(labels_a, labels_b)` where both are `sleap.Labels` instances 
             subsampled from these labels. 
  
         Notes: 
             If there is only 1 labeled frame, this will return two copies of the same 
             labels. For `len(labels) > 1`, splits are guaranteed to be mutually 
             exclusive. 
  
         Example: 
             You can generate multiple splits by calling this repeatedly: 
  
             ```py 
             # Generate a 0.8/0.1/0.1 train/val/test split. 
             labels_train, labels_val_test = labels.split(n=0.8) 
             labels_val, labels_test = labels_val_test.split(n=0.5) 
             ``` 
         """ 
         if len(self) == 1: 
             if copy: 
                 return self.copy(), self.copy() 
             else: 
                 return self, self 
  
         # Split indices. 
         if type(n) != int: 
             n = round(len(self) * n) 
         n = max(min(n, len(self) - 1), 1) 
         idx_a, idx_b = train_test_split(list(range(len(self))), train_size=n) 
  
         return self.extract(idx_a, copy=copy), self.extract(idx_b, copy=copy) 